In [1]:
# !pip install --quiet numpy==1.25.2 scipy tensorflow==2.14.1 tensorflow_federated==0.84.0
!pip uninstall -y numpy pandas
!pip install numpy==1.25.2 pandas==2.0.3 --force-reinstall --no-cache-dir


Found existing installation: numpy 1.25.2
Uninstalling numpy-1.25.2:
  Successfully uninstalled numpy-1.25.2
Found existing installation: pandas 2.0.3
Uninstalling pandas-2.0.3:
  Successfully uninstalled pandas-2.0.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 236.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 269.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 312.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 254.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 309.4 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    U

In [1]:
import numpy  as np

In [2]:
import pandas as pd


Load MovieLens 1M Data (Handles Encoding)

In [78]:
# Use ISO-8859-1 encoding as per dataset documentation and community advice[5][6]
users = pd.read_csv('/content/users.dat', sep='::', engine='python',
                    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'],
                    encoding='ISO-8859-1')
ratings = pd.read_csv('/content/ratings.dat', sep='::', engine='python',
                      names=['UserID', 'MovieID', 'Rating', 'Timestamp'],
                      encoding='ISO-8859-1')
movies = pd.read_csv('/content/movies.dat', sep='::', engine='python',
                     names=['MovieID', 'Title', 'Genres'],
                     encoding='ISO-8859-1')

# Map user and movie IDs to indices
user_id_map = {uid: idx for idx, uid in enumerate(users['UserID'].unique())}
movie_id_map = {mid: idx for idx, mid in enumerate(movies['MovieID'].unique())}

num_users = len(user_id_map)
num_movies = len(movie_id_map)

# Build binary user-movie request matrix
request_matrix = np.zeros((num_users, num_movies), dtype=np.int8)
for row in ratings.itertuples():
    uidx = user_id_map[row.UserID]
    midx = movie_id_map[row.MovieID]
    request_matrix[uidx, midx] = 1

print(f"Users: {num_users}, Movies: {num_movies}, Matrix shape: {request_matrix.shape}")


print("\nStart 5 Users:")
print(users.head())

print("\nStart 5 Ratings:")
print(ratings.head())

print("\nStart 5 Movies:")
print(movies.head())

print("\nStart 5 Rows of Request Matrix (Users × All Movies):")
print(request_matrix[:5])  # Rows: users 0–4, All movies

print("\nRequest Matrix: First 5 Users × First 10 Movies")
print(pd.DataFrame(request_matrix[:5, :10]))  # Preview in DataFrame

Users: 6040, Movies: 3883, Matrix shape: (6040, 3883)

Start 5 Users:
   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455

Start 5 Ratings:
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291

Start 5 Movies:
   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of th

Simulation Parameters and Device Initialization

In [97]:
# Simulation parameters from your table[3][4]
N = 10  # Number of devices
users_per_device = num_users // N  # num_users must be defined already
np.random.seed(42)

devices = []
for n in range(N):
    # Get user slice for device n
    start = n * users_per_device
    end = (n + 1) * users_per_device if n < N - 1 else num_users
    Xn = request_matrix[start:end]  # User-movie request submatrix for this device

    # Local training data and device characteristics
    Dn = np.random.randint(20, 51)                              # Number of samples
    wn = np.random.uniform(50, 100) * 8 * 1e6                   # Model upload size (in bits)
    alpha_n = np.random.uniform(18, 20.5) * 1e5                 # Data arrival rate
    Cn = np.random.randint(40, 61)                              # CPU cycles per sample
    Pn = np.random.uniform(0.01, 0.05)                          # Transmit power
    gamma_n = 10                                                # SNR (linear)

    # Cosine similarity (rho_n)
    if Xn.shape[0] > 1:
        sims = []
        for i in range(Xn.shape[0]):
            for j in range(i + 1, Xn.shape[0]):
                a, b = Xn[i], Xn[j]
                if np.linalg.norm(a) and np.linalg.norm(b):
                    sims.append(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))
        rho_n = np.mean(sims) if sims else 0.0
    else:
        rho_n = 0.0

    # Content request volume z_n
    z_n = Xn.sum()
    print(f"Device {n}: z_n = {z_n}")  # ✅ Print z_n for each device

    # Add device to the list
    devices.append({
        'Xn': Xn,
        'Dn': Dn,
        'wn': wn,
        'alpha_n': alpha_n,
        'Cn': Cn,
        'Pn': Pn,
        'gamma_n': gamma_n,
        'rho_n': rho_n,
        'z_n': z_n
    })

Device 0: z_n = 91373
Device 1: z_n = 105524
Device 2: z_n = 108351
Device 3: z_n = 98968
Device 4: z_n = 87065
Device 5: z_n = 104701
Device 6: z_n = 110051
Device 7: z_n = 101425
Device 8: z_n = 92011
Device 9: z_n = 100740


In [98]:
for i, dev in enumerate(devices):
    print(f"\nDevice {i} - Xn shape: {dev['Xn'].shape}")
    print(dev['Xn'])  # This prints the request matrix for device i
    print(f"Total movie requests (Xn.sum): {dev['Xn'].sum()}")


Device 0 - Xn shape: (604, 3883)
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]
Total movie requests (Xn.sum): 91373

Device 1 - Xn shape: (604, 3883)
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Total movie requests (Xn.sum): 105524

Device 2 - Xn shape: (604, 3883)
[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]
Total movie requests (Xn.sum): 108351

Device 3 - Xn shape: (604, 3883)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]]
Total movie requests (Xn.sum): 98968

Device 4 - Xn shape: (604, 3883)
[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 1]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Total movie requests (Xn.sum): 87065

Device 5 - Xn shape: (604, 3883)
[[0 0 0 .

Latency and Energy Calculation

In [99]:
def calc_latency_energy(dev):
    wS = 75 * 8 * 1e6  # 75 MB in bits
    Bdn_S = 150e6
    Bup_n = 100e6
    mu_n = 1
    f_n = 1e9
    beta_n = 1e-27
    P_dn_S = 1.0

    t_dn = wS / (Bdn_S * np.log2(1 + dev['gamma_n']))
    e_dn = P_dn_S * t_dn
    t_comp = mu_n * dev['Cn'] * dev['Dn'] / f_n
    e_comp = beta_n * mu_n * dev['Cn'] * dev['Dn'] * f_n ** 2
    t_up = dev['wn'] / (Bup_n * np.log2(1 + dev['gamma_n']))
    e_up = dev['Pn'] * t_up
    return t_dn + t_comp + t_up, e_dn + e_comp + e_up

for dev in devices:
    t_sec, e_joule = calc_latency_energy(dev)
    dev['t_n'] = t_sec * 1000     # Convert seconds → milliseconds
    dev['e_n'] = e_joule * 1000   # Optional: Convert joules → millijoules if your constraints are in mJ


Device Selection Optimization (Constraints 1a–1j)

In [6]:
!pip install cvxpy
!pip install ecos


Define Parameters

In [100]:
# Define simulation parameters from your constraint table
params = {
    'z_min': 800,
    'z_max': 7000,
    'D_max': 40,
    't_max': 1e-3,      # 1 ms
    'e_max': 1e-3,      # 1 mJ
    'S_min': 2,
    'S_max': 10         # Use 10 as per your table
}

print("Parameters defined:", params)


Parameters defined: {'z_min': 800, 'z_max': 7000, 'D_max': 40, 't_max': 0.001, 'e_max': 0.001, 'S_min': 2, 'S_max': 10}


Optimisation problem

In [101]:
import cvxpy as cp
def device_selection_with_offloading(devices, params):
    N = len(devices)
    # Decision variables
    s = cp.Variable(N, boolean=True)           # Device selection
    phi = cp.Variable((N, N), nonneg=True)     # Offloading fractions
    t = cp.Variable(nonneg=True)               # Round latency

    # Device parameters
    z = np.array([d['z_n'] for d in devices])
    rho = np.array([d['rho_n'] for d in devices])
    D = np.array([d['Dn'] for d in devices])
    t_n = np.array([d['t_n'] for d in devices])
    e_n = np.array([d['e_n'] for d in devices])
    alpha = np.array([d['alpha_n'] for d in devices])

    # Normalize rho for numerical stability
    rho_norm = rho / (np.max(rho) + 1e-10)

    # Objective: maximize sum(s_n * z_n * rho_n)
    objective = cp.Maximize(cp.sum(cp.multiply(s, z * rho_norm)))

    constraints = []

    # 1a: Variable bounds (handled by cp.Variable)
    constraints += [phi <= 1]

    # 1b: s_n*z_n + (1-s_n)*z_min >= z_min
    constraints += [cp.multiply(s, z) + cp.multiply(1 - s, params['z_min']) >= params['z_min']]

    # 1c: (1-s_n)*z_n + alpha_n*t <= z_max
    constraints += [cp.multiply(1 - s, z) + cp.multiply(alpha, t) <= params['z_max']]

    # 1d: s_n*Dn <= D_max
    constraints += [cp.multiply(s, D) <= params['D_max']]

    # 1e: t <= t_max, t >= max(s_n * t_n)
    constraints += [t <= params['t_max']]
    constraints += [t >= cp.max(cp.multiply(s, t_n))]

    # 1f: s_n*e_n <= e_max
    constraints += [cp.multiply(s, e_n) <= params['e_max']]

    # 1g: S_min <= sum(s_n) <= S_max
    constraints += [cp.sum(s) >= params['S_min']]
    constraints += [cp.sum(s) <= params['S_max']]

    # 1h: sum_m phi_mn <= 1 for all n
    constraints += [cp.sum(phi, axis=0) <= 1]

    # 1i: (1-s_n)(1-s_m)phi_nm = 0  (linearized: phi_nm <= s_n + s_m)
    for n in range(N):
        for m in range(N):
            if n != m:
                constraints.append(phi[n, m] <= s[n] + s[m])

    # 1j: s_n * phi_nm = 0 (linearized: phi_nm <= 1 - s_n)
    for n in range(N):
      for m in range(N):
         if n != m:
            # 1i: Two unselected devices can't talk
            constraints.append(phi[n, m] <= s[n] + s[m])
            # 1j: Selected device can't offload
            constraints.append(phi[n, m] <= 1 - s[n])

    # New: Only selected devices can receive offloaded data
    for m in range(N):
      constraints.append(cp.sum(phi[:, m]) <= s[m])
    # NEW: Prevent self-offloading
    for n in range(N):
       constraints.append(phi[n, n] == 0)

    # Solve the problem
    prob = cp.Problem(objective, constraints)

    try:
        prob.solve(solver=cp.ECOS_BB, verbose=False)

        if prob.status == cp.OPTIMAL:
            selected = (s.value > 0.5).astype(int)
            phi_val = phi.value
            print(f"Optimization status: {prob.status}, Objective: {prob.value}")
            print(f"Number of selected devices: {np.sum(selected)}")
        else:
            print("Fallback: Optimization was not optimal.")
            print(f"Status: {prob.status}")
            # Fallback: greedy selection
            scores = z * rho_norm
            selected_idx = np.argsort(scores)[-params['S_max']:]
            selected = np.zeros(N, dtype=int)
            selected[selected_idx] = 1
            phi_val = np.zeros((N, N))

    except Exception as e:
        print(f"Optimization failed: {e}")
        # Fallback: greedy selection
        scores = z * rho_norm
        selected_idx = np.argsort(scores)[-params['S_max']:]
        selected = np.zeros(N, dtype=int)
        selected[selected_idx] = 1
        phi_val = np.zeros((N, N))

    return selected, phi_val



Lets Test

In [74]:
# # Simulate 3 devices: 1 weak, 1 strong, 1 average
# devices = [
#     {'z_n': 1000, 'rho_n': 0.01, 'Dn': 20, 't_n': 0.9, 'e_n': 0.9, 'alpha_n': 1.5e5},  # Weak
#     {'z_n': 100000, 'rho_n': 0.95, 'Dn': 30, 't_n': 0.1, 'e_n': 0.1, 'alpha_n': 1.5e5}, # Strong
#     {'z_n': 50000, 'rho_n': 0.5, 'Dn': 25, 't_n': 0.5, 'e_n': 0.5, 'alpha_n': 1.5e5},   # Medium
# ]

# params = {
#     'z_min': 10000,
#     'z_max': 200000,
#     'D_max': 40,
#     't_max': 1.0,
#     'e_max': 1.0,
#     'S_min': 1,
#     'S_max': 2
# }


device_selection_with_offloading

In [102]:
selected, phi_val = device_selection_with_offloading(devices, params)

print("Selected Devices:", selected)
print("\nOffloading Matrix (phi):")
print(np.round(phi_val, 2))

# Show which devices sent or received data
for n in range(len(devices)):
    for m in range(len(devices)):
        if phi_val[n, m] > 1e-4:
            print(f"Device {n} offloaded {phi_val[n, m]:.2f} fraction to Device {m}")

# Print all device info for clarity
print("\nDevice Stats:")
for i, d in enumerate(devices):
    print(f"Device {i}: z={d['z_n']}, rho={d['rho_n']:.2f}, D={d['Dn']}, t={d['t_n']:.2f}, e={d['e_n']:.2f}, selected={selected[i]}")



Fallback: Optimization was not optimal.
Status: infeasible
Selected Devices: [1 1 1 1 1 1 1 1 1 1]

Offloading Matrix (phi):
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

Device Stats:
Device 0: z=91373, rho=0.12, D=26, t=3233.53, e=1226.78, selected=1
Device 1: z=105524, rho=0.12, D=26, t=2828.02, e=1230.90, selected=1
Device 2: z=108351, rho=0.11, D=23, t=2477.71, e=1220.74, selected=1
Device 3: z=98968, rho=0.11, D=31, t=3397.73, e=1234.04, selected=1
Device 4: z=87065, rho=0.12, D=45, t=2919.28, e=1210.90, selected=1
Device 5: z=104701, rho=0.12, D=50, t=2473.81, e=1181.71, selected=1
Device 6: z=110051, rho=0.11, D=49, t=3220.39, e=1225.82, selected=1
Device 7: z=101425, rho=0.11, D=22, t=3306.83, e=